# 3단계: 모델 구축 CI/CD 파이프라인 추가

이 단계에서는 [Amazon SageMaker Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects.html)를 사용하여 모델 구축을 위한 자동화된 CI/CD 파이프라인을 생성합니다.

![](img/sagemaker-mlops-project-build.jpg)

[모델 구축 및 훈련을 위한 SageMaker 제공 MLOps 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-code-commit)을 사용하여 [AWS CodePipeline](https://aws.amazon.com/codepipeline/)과 [AWS CodeCommit](https://aws.amazon.com/codecommit/) 코드 저장소를 통한 CI/CD 워크플로우 자동화를 프로비저닝할 것입니다.

SageMaker 프로젝트 템플릿은 다음과 같은 코드 저장소, 워크플로우 자동화 도구 및 파이프라인 단계 선택을 제공합니다:
- **코드 저장소**: AWS CodeCommit 또는 GitHub 및 Bitbucket과 같은 타사 Git 저장소
- **CI/CD 워크플로우 자동화**: AWS CodePipeline 또는 Jenkins
- **파이프라인 단계**: 모델 구축 및 훈련, 모델 배포 또는 둘 다

<div class="alert alert-info"> 이 노트북에서는 JupyterLab의 <code>Python 3</code> 커널을 사용하고 있는지 확인하세요.</div>

In [ ]:
import boto3
import sagemaker 
import json
from time import gmtime, strftime, sleep
from IPython.display import HTML

## MLOps 프로젝트 생성

### 준비

<div class="alert alert-info"><b>사전 요구 사항:</b> 아래 설명된 대로 AWS 계정에서 GitHub 사용자 또는 조직으로 AWS CodeStar 연결을 설정해야 합니다.<br/><b>이 AWS CodeStar 연결에 키가 <code>sagemaker</code>이고 값이 <code>true</code>인 태그를 추가하세요.</b></div>

자세한 지침은 SageMaker 개발자 가이드의 [타사 Git 저장소를 사용한 SageMaker AI MLOps 프로젝트 살펴보기](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)를 참조하세요.

<div class="alert alert-info">GitHub 계정에 두 개의 새로운 빈 프라이빗 저장소를 생성해야 합니다. 하나는 모델 빌드 파이프라인용이고 다른 하나는 모델 배포 파이프라인용입니다. 예를 들어, 저장소 이름을 <code>model-build</code>와 <code>model-deploy</code>로 지정할 수 있습니다.</div>

#### GitHub 연결 설정

접근하고 [AWS CodeConnection 연결](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html)을 통해 연결할 수 있는 GitHub 개인 계정이 필요합니다.

다음 지침에 따라 GitHub에 연결하세요(자세한 내용은 [문서](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-connect-3rdgit)를 참조하세요).

먼저 GitHub 개인 계정에 두 개의 빈 저장소를 생성합니다. 이 두 저장소는 모델 빌드 워크플로우와 모델 배포 워크플로우를 위한 시드 코드로 채워질 것입니다. [GitHub.com](https://github.com/)으로 이동하여 사용자 드롭다운 메뉴에서 Your repositories로 이동하세요.
model-build 저장소를 생성하세요.

![](img/github-model-build.png)

model-deploy 저장소에 대해서도 동일한 과정을 반복하세요.

![](img/github-model-deploy.png)

이제 [Connections](https://docs.aws.amazon.com/dtconsole/latest/userguide/welcome-connections.html)(AWS 내 개발자 도구의 일부)를 통해 개인 GitHub 계정을 AWS 계정에 연결하여 코드 변경 시 워크플로우 트리거를 활성화해야 합니다. [Connections](https://console.aws.amazon.com/codesuite/settings/connections)로 이동하여 새 연결을 생성하세요.

![](img/create-connection.png)

이제 GitHub를 제공자로 선택하고, 이름(예: `mlops-connection`)을 지정한 다음, 키가 sagemaker이고 값이 true인 새 태그를 추가하세요.

![](img/github-connection.png)

GitHub 계정에 인증하라는 메시지가 표시됩니다. 모범 사례로, 새 앱을 설치하여 방금 생성한 model-build 및 model-deploy 두 저장소에만 권한을 제한하는 것이 좋습니다.

![](img/install-github-app.png)
<img src="img/connection-permissions.png" width="500" height="800" alt="Description">

연결을 생성하고 ARN을 아래 셀에 복사 붙여넣기 하세요.
참고: 이 전체 과정은 빈 저장소를 생성하는 한 지원되는 모든 제공자에 대해 수행할 수 있습니다.

In [ ]:
# set this variable to the ARN of your code connection your created
code_connection_arn = "<CODESTAR ARN>"

⭐ 이 노트북에서 프로그래밍 방식으로 프로젝트를 생성할 수 있습니다 - **옵션 1** 또는 Studio UI에서 - **옵션 2**.

옵션 1은 수동 입력이 필요하지 않고 UX에 의존하지 않기 때문에 권장됩니다.</br>
옵션 2는 [**프로젝트 생성** UI 흐름](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-create.html)을 보여주기 위해 제공됩니다.

### 옵션 1: 프로그래밍 방식으로 프로젝트 생성
이 섹션에서는 `boto3`를 사용하여 SageMaker API를 통해 MLOps 프로젝트를 생성합니다.

In [ ]:
sm = boto3.client("sagemaker")
sc = boto3.client("servicecatalog")

sc_provider_name = "Amazon SageMaker"
sc_product_name = "MLOps template for model building, training, and deployment with third-party Git repositories using CodePipeline"

In [ ]:
p_ids = [p['ProductId'] for p in sc.search_products(
    Filters={
        'FullTextSearch': [sc_product_name]
    },
)['ProductViewSummaries'] if p["Name"]==sc_product_name]

In [ ]:
p_ids

In [ ]:
# If you get any exception from this code, go to the Option 2 and create a project in Studio UI
if not len(p_ids):
    raise Exception("No Amazon SageMaker ML Ops products found!")
elif len(p_ids) > 1:
    raise Exception("Too many matching Amazon SageMaker ML Ops products found!")
else:
    product_id = p_ids[0]
    print(f"ML Ops product id: {product_id}")

In [ ]:
# output what this project is about
sc.describe_product(Id=product_id)['ProductViewSummary']['ShortDescription']

In [ ]:
# get the latest template version
provisioning_artifact_id = sorted(
    [i for i in sc.list_provisioning_artifacts(
        ProductId=product_id
    )['ProvisioningArtifactDetails'] if i['Guidance']=='DEFAULT'],
    key=lambda d: d['Name'], reverse=True)[0]['Id']

In [ ]:
provisioning_artifact_id

In [ ]:
project_name = f"model-build-{strftime('%m-%d-%H-%M-%S', gmtime())}"

프로젝트 템플릿의 매개변수를 설정합니다. `model_build_code_repository_full_name` 및 `model_deploy_code_repository_full_name` 변수를 GitHub 저장소의 전체 이름으로 설정하세요. 이들은 새로운 빈 저장소여야 합니다.

In [ ]:
# Branch name for the Model Building and Training Code Repository
model_build_code_repository_branch = 'main'
# Full repository name of the Model Building and Training Code Repository, which would be username/reponame or organizationname/reponame
model_build_code_repository_full_name = "<BUILD GITHUB REPONAME>" # e.g. "username/reponame"

# Branch name for the Model Deployment Code Repository
model_deploy_code_repository_branch = 'main'
# Full repository name of the Model Deployment Code Repository, which would be username/reponame or organizationname/reponame
model_deploy_code_repository_full_name = "<DEPLOY GITHUB REPONAME>" # e.g. "username/reponame"

In [ ]:
# set project parameters
project_parameters = [
    {
        'Key': 'ModelBuildCodeRepositoryBranch',
        'Value': model_build_code_repository_branch,
    },
    {
        'Key': 'ModelBuildCodeRepositoryFullname',
        'Value': model_build_code_repository_full_name,
    },
    {
        'Key': 'ModelDeployCodeRepositoryBranch',
        'Value': model_deploy_code_repository_branch,
    },
    {
        'Key': 'ModelDeployCodeRepositoryFullname',
        'Value': model_deploy_code_repository_full_name,
    },
        {
        'Key': 'CodeConnectionArn',
        'Value': code_connection_arn,
    }
]

마지막으로, 서비스 카탈로그 제품 템플릿에서 SageMaker 프로젝트를 생성합니다:

In [ ]:
print(f'''Creating a {project_name} using {sc_product_name} with the following parameters:
{json.dumps(project_parameters, indent=2)}
''')

In [ ]:
# create SageMaker project
r = sm.create_project(
    ProjectName=project_name,
    ProjectDescription="Model build and deploy project",
    ServiceCatalogProvisioningDetails={
        'ProductId': product_id,
        'ProvisioningArtifactId': provisioning_artifact_id,
        'ProvisioningParameters': project_parameters
    },
)

print(r)
project_id = r["ProjectId"]
%store project_id

<div class="alert alert-info"> 💡 <strong> 다음 셀을 실행하여 프로젝트 생성이 완료될 때까지 기다리세요</strong>
</div>

In [ ]:
# Project creation takes about 3-5 min
while sm.describe_project(ProjectName=project_name)['ProjectStatus'] != 'CreateCompleted':
    print("Waiting for project creation completion")
    sleep(10)
    
print(f"MLOps project {project_name} creation completed")
%store project_name

<img src="

### End of Option 1: Create project programmatically
Now you have provisioned a project template in your SageMaker environment. Navigate to the section **Configure the MLOps project**.

---

### Option 2: Create a project in Studio UI
<div class="alert alert-info"> 💡 <strong> Skip this section if you created a project programmatically </strong>

Follow the [instructions for **Step 2: Create the Project**](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html#sagemaker-proejcts-walkthrough-create-3rdgit) in the Developer Guide to create an MLOps CI/CD project.

For the template choose the **Model building, training, and deployment with third-party Git repositories using CodePipeline**:

![](img/mlops-project-name.png)

Wait until project is created.

### Resolve issues with project creation

#### Project creation process stuck in pending
If after 5 minutes the project creation banner is still on, close the Studio browser window and sign in Studio again.

![](img/project-creation-pending.png)

#### Error messages
❗ If you see an error message similar to:
```
Your project couldn't be created
Studio encountered an error when creating your project. Try recreating the project again.

CodeBuild is not authorized to perform: sts:AssumeRole on arn:aws:iam::XXXX:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole (Service: AWSCodeBuild; Status Code: 400; Error Code: InvalidInputException; Request ID: 4cf59a54-0c59-476a-a970-0ac656db4402; Proxy: null)
```

see steps 5-6 of [SageMaker Studio Permissions Required to Use Projects](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-studio-updates.html). Make sure you have all required project roles listed in the **Apps** card under **Projects**. 

Alternatively, you can create the required roles by using the provided CloudFormation template [`cfn-templates/sagemaker-project-templates-roles.yaml`](cfn-templates/sagemaker-project-templates-roles.yaml). 
Run in the repository clone directory from the command line terminal where you have the corresponding permissions:

```sh
aws cloudformation deploy \
    --template-file cfn-templates/sagemaker-project-templates-roles.yaml \
    --stack-name sagemaker-project-template-roles \
    --capabilities CAPABILITY_IAM CAPABILITY_NAMED_IAM \
    --parameter-overrides \
    CreateCloudFormationRole=YES \
    CreateCodeBuildRole=YES \
    CreateCodePipelineRole=YES \
    CreateEventsRole=YES \
    CreateProductsExecutionRole=YES 
```

### End of Option 2: Create a project in Studio UI
Now when you have the project created, move to the section **Configure the MLOps project**.

---

## Configure the MLOps project
The provisioned MLOps project implements two CodePipeline pipelines. One is for model building and another for model deployment. The project also populates two GitHub repositories your specified with seed code. This notebook explains and configures the model building pipeline. The next notebook `05-deploy` deals with the second part - the model deployment pipeline.

### Model building pipeline

The project provisions and runs a default model building pipeline automatically as soon as it has been created. This pipeline is a sample placeholder in the project for your own custom pipeline. Ignore the default pipeline for the moment.
The project templates deploys the following resources in your AWS account:

![](img/mlops-model-build-train.png)

The main components are:
1. The project template is made available through SageMaker Projects and AWS Service Catalog portfolio
2. A CodePipeline pipeline with two stages - `Source` to download the source code from your GitHub repository and `Build` to create and execute a SageMaker pipeline
3. A default SageMaker pipeline with model build, train, and register workflow
4. A seed code repository in GitHub with a provided default version of a seed code

이 프로젝트는 사전 정의된 템플릿에서 자동화된 CI/CD 파이프라인을 구현하는 데 필요한 모든 코드와 인프라를 포함하고 있습니다.
파이프라인과 함께 프로젝트를 시작하려면 다음 단계를 완료해야 합니다:
1. 프로젝트 GitHub 저장소를 노트북 로컬 스토리지에 복제합니다
2. ML 파이프라인 템플릿 샘플 코드를 3단계 노트북에서 구현한 실제 파이프라인 구성 코드로 교체합니다
3. 올바른 Python 모듈 이름을 참조하고 프로젝트 매개변수를 설정하도록 `codebuild-buildspec.yml` 파일을 수정합니다

다음 섹션에서는 이러한 단계를 안내합니다. 자세한 지침과 실습 예제는 개발 가이드 [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough-3rdgit.html)를 참조하세요.

MLOps 프로젝트를 생성하기 위해 옵션 1 `boto3`를 사용한 경우, `project_name`과 `project_id`가 자동으로 설정됩니다. 다음 코드 셀을 실행하여 값을 출력할 수 있습니다. UI 지침에 따라 프로젝트를 생성한 경우 `project_name`을 수동으로 설정해야 합니다.

In [ ]:
try:
    print(project_name)
    print(project_id)
    print(model_build_code_repository_full_name)
    print(code_connection_arn)
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("Set the code_connection_arn, project_name, and repository full name in the following code cell")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
# project_name = "<SET TO THE NAME OF THE CREATED PROJECT>" # Keep commented out if you used option 1 to create a project
# model_build_code_repository_full_name = "<SET TO THE FULL NAME OF MODEL BUILD REPO>" # Keep commented out if you used option 1 to create a project

region = sm.meta.region_name
r = sm.describe_project(ProjectName=project_name)
project_id = r['ProjectId']
project_arn = r['ProjectArn']
repository_name = model_build_code_repository_full_name.split('/')[1]
git_folder = project_name
project_folder = f'sagemaker-{project_id}-modelbuild'
project_path = f'{git_folder}/{project_folder}'

#%store project_name
#%store project_arn
#%store project_id
    
print(f"Project path: {project_path}")
print(f"Project folder: {project_folder}")

### 1. 프로젝트 시드 코드를 JupyterLab 파일 시스템에 복제하기
JupyterLab 터미널과 GitHub CLI를 사용하여 GitHub 저장소에서 프로젝트 코드를 복제해야 합니다.

1. **File** > **New** > **Terminal**을 통해 새 터미널 창을 엽니다
2. 다음 셀의 출력을 복사하여 터미널 명령줄에 붙여넣습니다
3. `codeconnections` 자격 증명 도우미와 함께 `git clone`을 사용하여 모델 빌드 저장소를 복제합니다.

In [ ]:
def copy_output(text):
    return HTML(f'''
        <div style="position: relative;">
            <pre>{text}</pre>
            <button onclick="navigator.clipboard.writeText(`{text}`)"
                    style="position: absolute; top: 5px; right: 5px;">
                Copy
            </button>
        </div>
    ''')
    
cmd = f'''
git config --global credential.UseHttpPath true
git config --global credential.helper 'cache --timeout=720000'
git config --global credential.helper '!aws codecommit credential-helper $@'

mkdir -p $HOME/{project_path}

git clone https://codeconnections.{region}.amazonaws.com/git-http/{code_connection_arn.split(':')[4]}/{region}/{code_connection_arn.split(':')[-1].split('/')[-1]}/{model_build_code_repository_full_name}.git  $HOME/{project_path}
'''

copy_output(cmd)

### 2. 파이프라인 구성 코드 교체하기

템플릿 코드가 포함된 프로젝트를 사용자 지정하려면 다음 단계가 필요합니다. 다음 코드 셀은 필요한 모든 단계를 실행하므로 수동으로 아무것도 할 필요가 없습니다. 다음 텍스트는 참고용입니다.

- 소스 코드는 `sagemaker-<project-name>-<project-id>-modelbuild` 폴더에 있습니다.
- 원본 파일 `codebuild-buildspec.yml`은 `codebuild-buildspec-original.yml`로 이름이 변경됩니다.
- 파이프라인 코드가 포함된 프로젝트 코드 저장소 폴더는 `abalone` 폴더에서 `playerchurn`으로 이름이 변경됩니다.
- 템플릿 파이프라인이 있는 원본 파일 `pipeline.py`는 `pipeline-original.py`로 이름이 변경됩니다.
- `pipeline_steps` Python 모듈을 프로젝트 코드 저장소 폴더의 `pipelines` 폴더에 복사합니다.
- 노트북 3에서 생성된 `requirements.txt`를 프로젝트 코드 저장소 폴더의 `pipelines` 폴더에 복사합니다.
- 노트북 3에서 SageMaker Python SDK 기본 구성 파일 `config.yaml`을 프로젝트 코드 저장소 폴더의 `pipelines` 폴더에 복사합니다.

In [ ]:
# see the workshop folder name
!pwd

In [ ]:
# if you local path for the workshop folder is different, set the correct absolute path to the variable workshop_folder
workshop_folder = "mlops-sagemaker-mlflow"

In [ ]:
!mkdir -p ~/{workshop_folder}/pipelines
!mv ~/{project_path}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec-original.yml
!mv ~/{project_path}/pipelines/abalone ~/{project_path}/pipelines/playerchurn
!cp ~/{workshop_folder}/requirements.txt ~/{project_path}
!cp ~/{workshop_folder}/config.yaml ~/{project_path}

모든 것이 작동하는지 확인하기 위해 원격으로 실행하기 전에 파이프라인을 로컬에서 테스트합니다.

In [ ]:
# Variables to be used in the CICD build pipeline.
region = "us-east-1"
feature_group_name = "<FEATURE GROUP NAME>" # replace feature group with the one created in the previous lab.
bucket_name = "<sagemaker-region-accountid>" # replace the S3 bucket name with the one created for this workshop. i.e sagemaker-[region]-[aws account id]
bucket_prefix = "player-churn/xgboost"
experiment_name = "player-churn-model-build-pipeline"
train_instance_type = "ml.m5.xlarge"
test_score_threshold = 0.75
model_package_group_name = f"{project_name}-{project_id}"
model_approval_status = "PendingManualApproval"
mlflow_tracking_server_arn = "<MLFLOW ARN>" # Provide a valid mlflow tracking server ARN. You can find the value in the output from 00-start-here.ipynb
pipeline_name = "<PIPELINE NAME>" # replace the value with the name of the training pipeline if it is different from the value given in previous lab.

In [ ]:
assert len(feature_group_name) > 0
assert len(bucket_name) > 0
assert len(mlflow_tracking_server_arn) > 0
assert len(pipeline_name) > 0

In [ ]:
%store model_package_group_name
%store region
%store bucket_name
%store bucket_prefix

In [ ]:
# Update pipeline parameters

# Get current pipeline definition
current_pipeline = sm.describe_pipeline(PipelineName=pipeline_name)
current_definition = json.loads(current_pipeline['PipelineDefinition'])

# Define your required parameters
required_parameters = {
    "region": region,
    "feature_group_name": feature_group_name,
    "bucket_name": bucket_name,
    "bucket_prefix": bucket_prefix,
    "experiment_name": experiment_name,
    "train_instance_type": train_instance_type,
    "test_score_threshold": str(test_score_threshold),
    "model_package_group_name": model_package_group_name,
    "model_approval_status": model_approval_status,
    "mlflow_tracking_server_arn": mlflow_tracking_server_arn
}

# Ensure Parameters section exists
if 'Parameters' not in current_definition:
    current_definition['Parameters'] = []

# Create a set of existing parameter names for quick lookup
existing_params = {param['Name']: param for param in current_definition['Parameters']}

# Add or update each required parameter
for param_name, param_value in required_parameters.items():
    if param_name in existing_params:
        # Update existing parameter
        old_value = existing_params[param_name].get('DefaultValue', 'No default')
        existing_params[param_name]['DefaultValue'] = param_value
        print(f"Updated '{param_name}': '{old_value}' -> '{param_value}'")
    else:
        # Add new parameter
        new_param = {
            'Name': param_name,
            'Type': 'String',
            'DefaultValue': param_value
        }
        current_definition['Parameters'].append(new_param)
        print(f"Added '{param_name}': '{param_value}'")

# Update the pipeline
sm.update_pipeline(
    PipelineName=pipeline_name,
    PipelineDefinition=json.dumps(current_definition),
    RoleArn=current_pipeline['RoleArn']
)

print(f"\nPipeline {pipeline_name} successfully updated with all required parameters!")


In [ ]:
# Start pipeline execution

response = sm.start_pipeline_execution(
    PipelineName=pipeline_name,
    PipelineParameters=[
        {
            'Name': 'region',
            'Value': region
        },
        {
            'Name': 'feature_group_name',
            'Value': feature_group_name
        },
        {
            'Name': 'bucket_name',
            'Value': bucket_name
        },
        {
            'Name': 'bucket_prefix',
            'Value': bucket_prefix
        },
        {
            'Name': 'experiment_name',
            'Value': experiment_name
        },
        {
            'Name': 'train_instance_type',
            'Value': train_instance_type
        },
        {
            'Name': 'test_score_threshold',
            'Value': str(test_score_threshold)
        },
        {
            'Name': 'model_package_group_name',
            'Value': model_package_group_name
        },
        {
            'Name': 'model_approval_status',
            'Value': model_approval_status
        },
        {
            'Name': 'mlflow_tracking_server_arn',
            'Value': mlflow_tracking_server_arn
        }
    ],
    
)

파이프라인이 완료될 때까지 기다립니다.

In [ ]:
pipeline_exec_id = response["PipelineExecutionArn"]
describe_pipeline_response = sm.describe_pipeline_execution(
    PipelineExecutionArn=pipeline_exec_id
)

pipeline_execution_status = describe_pipeline_response["PipelineExecutionStatus"]
while True:
   if pipeline_execution_status in ['Stopped', 'Failed', 'Succeeded']:
       print(f"Pipeline execution completed with status: {pipeline_execution_status}")
       break
   print(f"Pipeline execution status: {pipeline_execution_status}")
   sleep(10)
   describe_pipeline_response = sm.describe_pipeline_execution(PipelineExecutionArn=pipeline_exec_id)
   pipeline_execution_status = describe_pipeline_response["PipelineExecutionStatus"]

이 시점에서 파이프라인 구성 코드가 작동하고 파이프라인을 생성한다는 것을 로컬에서 테스트했습니다. Studio **Pipelines** 위젯에서 이 파이프라인을 볼 수 있습니다. 이제 CI/CD 파이프라인을 생성할 준비가 되었습니다.

#### 모델 패키지 그룹을 프로젝트에 연결하기
프로젝트 소유 리소스에는 비용 제어, 속성 기반 보안 제어 및 거버넌스를 위해 자동으로 `sagemaker:project-name` 및 `sagemaker:project-id` 태그가 지정됩니다.
모델 패키지 그룹이 모델 레지스트리에 이미 존재하므로 이 프로젝트에 연결하기 위해 태그를 지정해야 합니다. 다음 코드 셀은 `AddTags` API를 호출하여 모델 패키지 그룹에 프로젝트 태그를 설정합니다.

In [ ]:
model_package_group_arn = sm.describe_model_package_group(ModelPackageGroupName=model_package_group_name).get("ModelPackageGroupArn")

if model_package_group_arn:
    print(f"Adding tags {project_arn.split('/')[-1]} and {project_id} for model package group {model_package_group_arn}")
    r = sm.add_tags(
        ResourceArn=model_package_group_arn,
        Tags=[
            {
                'Key': 'sagemaker:project-name',
                'Value': project_arn.split("/")[-1]
            },
            {
                'Key': 'sagemaker:project-id',
                'Value': project_id
            },
        ]
    )
    print(r)
else:
    print(f"The model package group {model_package_group_name} doesn't exist")
    
sm.list_tags(ResourceArn=model_package_group_arn)["Tags"]

### 3. 빌드 명세 파일 수정하기
다음 셀에서는 프로젝트 폴더의 `codebuild-buildspec.yml` 파일을 수정하여 파이프라인이 포함된 새 Python 모듈 이름을 반영하고 다른 프로젝트별 매개변수를 설정합니다.

파이프라인 생성 스크립트에 다음 매개변수를 전달해야 합니다:

- region - 파이프라인이 실행되는 리전
- feature_group_name - 특성 그룹의 이름
- bucket_name - 파이프라인에서 생성된 아티팩트를 저장하는 데 사용할 S3 버킷의 이름
- bucket_prefix - 아티팩트를 저장하기 위한 S3 버킷 접두사
- experiment_name - MLFlow 추적 서버에서 파이프라인 실행을 구성하기 위한 실험 이름
- train_instance_type - 모델 훈련에 사용할 인스턴스
- test_score_threshold - 모델을 레지스트리에 등록할지 평가하기 위한 최소 테스트 점수 임계값
- model_package_group_name - 모델 버전의 모델 패키지 그룹 이름
- model_approval_status - 모델이 Model Registry에 등록될 때의 기본 모델 승인 상태
- mlflow_tracking_server_arn - 유효한 MLFlow 추적 서버 ARN

In [ ]:
code_build_buildspec_template = r"""

version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.10
    commands:
      - pip install --upgrade --force-reinstall . "awscli>1.20.30"
      - pip install mlflow==2.13.2 sagemaker-mlflow s3fs xgboost
    
  build:
    commands:
      - export SAGEMAKER_USER_CONFIG_OVERRIDE="./config.yaml"
      - export PYTHONUNBUFFERED=TRUE
      - export SAGEMAKER_PROJECT_NAME_ID="${SAGEMAKER_PROJECT_NAME}-${SAGEMAKER_PROJECT_ID}"
      - |
        run-pipeline \
          --role-arn $SAGEMAKER_PIPELINE_ROLE_ARN \
          --tags "[{\"Key\":\"sagemaker:project-name\",\"Value\":\"${SAGEMAKER_PROJECT_NAME}\"}, {\"Key\":\"sagemaker:project-id\", \"Value\":\"${SAGEMAKER_PROJECT_ID}\"}]" \
          --pipeline-name "{{PIPELINE_NAME}}" \
          --kwargs "{ \
                \"region\":\"{{REGION}}\", \
                \"feature_group_name\":\"{{FEATURE_GROUP_NAME}}\",\
                \"bucket_name\":\"{{BUCKET_NAME}}\",\
                \"bucket_prefix\":\"{{BUCKET_PREFIX}}\",\
                \"experiment_name\":\"{{EXPERIMENT_NAME}}\", \
                \"train_instance_type\":\"{{TRAIN_INSTANCE_TYPE}}\", \
                \"test_score_threshold\":\"{{TEST_SCORE_THRESHOLD}}\",\
                \"model_package_group_name\":\"{{MODEL_PACKAGE_GROUP_NAME}}\",\
                \"model_approval_status\":\"{{MODEL_APPROVAL_STATUS}}\",\
                \"mlflow_tracking_server_arn\":\"{{MLFLOW_TRACKING_SERVER_ARN}}\"\
                    }"
      - echo "Create/update of the SageMaker Pipeline and a pipeline execution completed."
"""

In [ ]:
code_build_buildspec = code_build_buildspec_template.replace("{{REGION}}", region)
code_build_buildspec = code_build_buildspec.replace("{{PIPELINE_NAME}}", pipeline_name)
code_build_buildspec = code_build_buildspec.replace("{{FEATURE_GROUP_NAME}}", feature_group_name)
code_build_buildspec = code_build_buildspec.replace("{{BUCKET_NAME}}", bucket_name)
code_build_buildspec = code_build_buildspec.replace("{{BUCKET_PREFIX}}", bucket_prefix)
code_build_buildspec = code_build_buildspec.replace("{{EXPERIMENT_NAME}}", experiment_name)
code_build_buildspec = code_build_buildspec.replace("{{TRAIN_INSTANCE_TYPE}}", train_instance_type)
code_build_buildspec = code_build_buildspec.replace("{{TEST_SCORE_THRESHOLD}}", str(test_score_threshold))
code_build_buildspec = code_build_buildspec.replace("{{MODEL_PACKAGE_GROUP_NAME}}", model_package_group_name)
code_build_buildspec = code_build_buildspec.replace("{{MODEL_APPROVAL_STATUS}}", model_approval_status)
code_build_buildspec = code_build_buildspec.replace("{{MLFLOW_TRACKING_SERVER_ARN}}", mlflow_tracking_server_arn)

In [ ]:
with open("codebuild-buildspec.yml", "w") as f:
    f.write(code_build_buildspec)

워크숍 폴더에서 프로젝트 코드 저장소 폴더로 `codebuild-buildspec.yml` 파일을 복사합니다:

In [ ]:
!cp ~/{workshop_folder}/codebuild-buildspec.yml ~/{project_path}/codebuild-buildspec.yml
!cp ~/{workshop_folder}/run_pipeline.py ~/{project_path}/pipelines/run_pipeline.py

### 4. `setup.py` 파일 수정하기
마지막으로, 프로젝트 코드 저장소 폴더에서 `setup.py` 파일을 열고 `required_packages = ["sagemaker==2.XX.0"]` 줄을 `required_packages = ["sagemaker"]`로 바꿉니다. 변경 사항을 저장하세요.

왜 이 변경을 했나요? 고정된 sagemaker 라이브러리 버전은 버그이며 향후 기본 제공 SageMaker 프로젝트 템플릿 릴리스에서 수정될 예정입니다. 지금은 이 템플릿 파일을 수동으로 수정합니다. 기본 제공 프로젝트 템플릿은 편의를 위한 것이며 SageMaker 프로젝트 메커니즘을 사용하여 자체 사용자 지정 MLOps 프로젝트를 패키징하고 프로비저닝하는 방법을 보여주기 위한 것임을 기억하세요.

이제 CI/CD 모델 빌딩 파이프라인을 시작할 준비가 되었습니다.

CI/CD 파이프라인을 실행할 모든 준비가 완료되었습니다.

---

## 모델 빌딩 파이프라인을 위한 CI/CD 실행하기
모델 빌딩 파이프라인을 위한 CI/CD를 시작하려면 변경된 코드를 프로젝트 CodeCommit 저장소에 푸시해야 합니다.

<div class="alert alert-info">git 명령을 실행할 때 JupyterLab 터미널에서 저장소 코드가 포함된 폴더에 있는지 확인하세요. 폴더 이름은 <code>sagemaker-[project-name]-[project-id]-modelbuild</code>와 같습니다.</div>

아래 셀은 올바른 폴더 이름으로 필요한 `cd` 명령을 출력합니다:

In [ ]:
print(f"cd ~/{project_folder}")

JupyterLab 메뉴 **File** > **New** > **Terminal**을 통해 시스템 터미널 창을 열고 다음 명령을 입력하세요. `user.email`과 `user.name`을 유지하거나 귀하의 데이터로 바꾸세요.

In [ ]:
def copy_output(text):
    return HTML(f"""
        <div style="position: relative;">
            <pre>{text}</pre>
            <button onclick="copyText(this)" 
                    style="position: absolute; top: 5px; right: 5px;">
                Copy
            </button>
            <textarea id="hiddenText" style="position: absolute; left: -9999px;">{text}</textarea>
        </div>
        <script>
        function copyText(button) {{
            const textarea = button.parentNode.querySelector('#hiddenText');
            textarea.select();
            document.execCommand('copy');
            button.innerHTML = 'Copied!';
            setTimeout(() => button.innerHTML = 'Copy', 2000);
        }}
        </script>
    """)

cmd = f'''
cd ~/{project_path}

git config --global user.email "you@example.com"
git config --global user.name "Your Name"
  
git add -A
git commit -am "customize project"
git push
'''

copy_output(cmd)


## 새 모델 버전의 세부 정보 보기
파이프라인 실행이 완료된 후, 새 모델 버전이 모델 레지스트리에 등록되어야 합니다. 모델 버전 세부 정보를 보려면:

1. Studio 사이드바에서 **Models** 위젯을 선택합니다
2. 이전 랩에서 생성한 모델 패키지 그룹의 이름을 클릭하여 모델 그룹을 엽니다
3. 모델 버전 목록에서 최신 버전의 모델을 선택합니다

열리는 모델 버전 탭에서 활동, [모델 버전 세부 정보](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-details.html) 및 [데이터 계보](https://docs.aws.amazon.com/sagemaker/latest/dg/lineage-tracking.html)를 탐색할 수 있습니다.

![](img/model-version-details.png)

실제 프로젝트에서는 [모델 품질 메트릭](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html), [설명 가능성](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-model-explainability.html) 및 [편향](https://docs.aws.amazon.com/sagemaker/latest/dg/clarify-measure-post-training-bias.html) 보고서, 부하 테스트 데이터, [추론 추천자](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html)와 같은 다양한 모델 속성과 추가 모델 버전 메타데이터를 추가합니다.

Studio UI에서 모델 패키지 버전을 보려면 아래 코드 셀에서 구성한 링크를 클릭하세요. 파이프라인 실행이 완료될 때까지 기다려야 최신 등록된 모델 패키지 버전을 볼 수 있습니다.

## 요약
이 노트북에서는 다음 기능을 갖춘 CI/CD 파이프라인을 구현했습니다:
- 모델 빌딩 ML 파이프라인은 CodeCommit 저장소에서 소스 제어 하에 있습니다
- CodeCommit 저장소에 대한 모든 푸시는 ML 파이프라인을 구성, 업서트 및 실행하는 새로운 CodeBuild 빌드를 시작합니다
- 모델 빌딩 파이프라인을 포함한 전체 e2e 모델 개발 프로세스가 이제 자동화되었습니다
- SageMaker 프로젝트는 관련 ML 파이프라인, 저장소, 모델, 실험 및 추론 엔드포인트에 대한 메타데이터를 가진 Studio의 논리적 구성입니다

------

## 7단계 계속하기
7단계 [노트북](07-sagemaker-deploy.ipynb)을 엽니다.

## 다른

## Additional resources
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Enhance your machine learning development by using a modular architecture with Amazon SageMaker projects](https://aws.amazon.com/blogs/machine-learning/enhance-your-machine-learning-development-by-using-a-modular-architecture-with-amazon-sagemaker-projects/)
- [Dive deep into automating MLOps](https://www.youtube.com/watch?v=3_cHnk9VSfQ)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [`aws-samples` GitHub repository with custom project templates examples](https://github.com/aws-samples/sagemaker-custom-project-templates)

# 커널 종료

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>